In [7]:
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.utils import np_utils

In [8]:
image = np.loadtxt('bbs-train.txt')
label = np.loadtxt('label-train.txt')
image /= np.max(image)
# initialize the dataset
d = 4200
image_train = image[:d]
image_val = image[d:]
t = label[0:,1:]
t_x = np.reshape(t, (1,4546))
labels = np_utils.to_categorical(t_x, 2)
label_train = labels[:d]
label_val = labels[d:]

In [9]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [1]:
print(hex(24))

0x18


In [14]:

# Variables
x = tf.placeholder("float", [None, 800])
y_ = tf.placeholder("float", [None, 2])

w_enc = tf.Variable(tf.random_normal([800, 64], mean=0.0, stddev=0.05))
w_dec = tf.Variable(tf.random_normal([64, 800], mean=0.0, stddev=0.05))
# w_dec = tf.transpose(w_enc) # if you use tied weights
b_enc = tf.Variable(tf.zeros([64]))
b_dec = tf.Variable(tf.zeros([800]))

# Create the model
def model(X, w_e, b_e, w_d, b_d):
    encoded = tf.sigmoid(tf.matmul(X, w_e) + b_e)
    decoded = tf.sigmoid(tf.matmul(encoded, w_d) + b_d)
    
    return encoded, decoded

encoded, decoded = model(x, w_enc, b_enc, w_dec, b_dec)

# Cost Function basic term
cross_entropy = -1. * x * tf.log(decoded) - (1. - x) * tf.log(1. - decoded)
loss = tf.reduce_mean(cross_entropy)
train_step = tf.train.AdagradOptimizer(0.1).minimize(loss)

# Train
init = tf.global_variables_initializer()

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(decoded,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

with tf.Session() as sess:
    sess.run(init)
    print('Training...')
    for i in range(1000):
        batch_xs, batch_ys = next_batch(128, image_train, label_train)
        train_step.run({x: batch_xs, y_: batch_ys})
        
        if i % 100 == 0:
            train_loss = loss.eval({x: batch_xs, y_: batch_ys})
            print('  step, loss = %6d: %6.3f' % (i, train_loss))
        
    # generate decoded image with test data
    test_fd = {x: image_val, y_: label_val}
    decoded_imgs = decoded.eval(test_fd)
    print("training complete! ")
    print('loss (test) = ', loss.eval(test_fd))

Training...
  step, loss =      0:  0.697
  step, loss =    100:  0.555
  step, loss =    200:  0.447
  step, loss =    300:  0.364
  step, loss =    400:  0.305
  step, loss =    500:  0.252
  step, loss =    600:  0.216
  step, loss =    700:  0.183
  step, loss =    800:  0.165
  step, loss =    900:  0.136
training complete! 
loss (test) =  0.117642
